In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Bottleneck Block for ResNet-101
class Bottleneck(nn.Module):
    expansion = 4  # Output channels are expanded

    def __init__(self, in_channels, mid_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)

        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)

        self.conv3 = nn.Conv2d(mid_channels, mid_channels * self.expansion,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(mid_channels * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# Build ResNet class
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):  # CIFAR-10 has 10 classes
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual layers
        self.layer1 = self._make_layer(block, 64,  layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, mid_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != mid_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, mid_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(mid_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, mid_channels, stride, downsample))
        self.in_channels = mid_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, mid_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

# ResNet-101 architecture has [3, 4, 23, 3] Bottleneck blocks
resnet101 = ResNet(Bottleneck, [3, 4, 23, 3]).to(device)

# Show model summary for CIFAR-10 image size (3, 224, 224)
summary(resnet101, input_size=(3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [3]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
# 🔄 Data augmentation and normalization for training
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),  # ResNet-101 requires 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

# ✅ Standard normalization for test set (no augmentation)
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

# 📥 Load CIFAR-10 datasets
train_dataset = datasets.CIFAR10(root='./data', train=True,
                                 download=True, transform=train_transform)

test_dataset = datasets.CIFAR10(root='./data', train=False,
                                download=True, transform=test_transform)

# 🚚 DataLoaders
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# 🔍 Dataset sizes
print(f"Train size: {len(train_dataset)} samples")
print(f"Test size: {len(test_dataset)} samples")


100%|██████████| 170M/170M [00:14<00:00, 11.6MB/s]


Train size: 50000 samples
Test size: 10000 samples


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

# ✅ Loss Function
criterion = nn.CrossEntropyLoss()

# ✅ Optimizer: Adam (can replace with SGD)
optimizer = optim.Adam(resnet101.parameters(), lr=0.001, weight_decay=1e-4)

# ✅ Scheduler: StepLR (reduce LR every 20 epochs)
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

# ✅ Training Configuration
num_epochs = 50  # You can increase to 100 or more if resources allow
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet101 = resnet101.to(device)

# ✅ Initialize Metric Lists
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# ✅ Confirm Setup
print({
    "loss_function": criterion.__class__.__name__,
    "optimizer": optimizer.__class__.__name__,
    "scheduler": scheduler.__class__.__name__,
    "epochs": num_epochs,
    "device": device.type
})


{'loss_function': 'CrossEntropyLoss', 'optimizer': 'Adam', 'scheduler': 'StepLR', 'epochs': 50, 'device': 'cuda'}


In [6]:
# 🔍 Evaluate model on test/validation data
def evaluate_model(model, dataloader, criterion):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / total
    accuracy = 100 * correct / total
    return avg_loss, accuracy


In [8]:
best_val_acc = 0.0

In [9]:
for epoch in range(num_epochs):
    resnet101.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward
        outputs = resnet101(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Epoch summary
    train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # 🔍 Evaluate on test set
    val_loss, val_acc = evaluate_model(resnet101, test_loader, criterion)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    # 🔁 Step scheduler
    scheduler.step()

    # 💾 Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(resnet101.state_dict(), "best_resnet101.pth")
        print(f"✅ Best model updated at epoch {epoch+1} — Val Acc: {val_acc:.2f}%")

    # 🖨️ Logging
    print(f"[Epoch {epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% || "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")


✅ Best model updated at epoch 1 — Val Acc: 61.04%
[Epoch 1/50] Train Loss: 0.9906 | Train Acc: 64.43% || Val Loss: 1.1714 | Val Acc: 61.04%


KeyboardInterrupt: 

In [ ]:
plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies)


In [ ]:
resnet101.load_state_dict(torch.load("best_resnet101.pth"))
resnet101.eval()


In [ ]:
import matplotlib.pyplot as plt
import torch

# 🔍 Evaluation function
def evaluate_model(model, dataloader, criterion):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = total_loss / total
    accuracy = 100 * correct / total
    return avg_loss, accuracy


In [ ]:
def plot_metrics(train_losses, val_losses, train_acc, val_acc):
    epochs = range(1, len(train_losses)+1)

    plt.figure(figsize=(12, 5))

    # 🔹 Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, val_losses, 'r--', label='Validation Loss')
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # 🔹 Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_acc, 'b-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'r--', label='Validation Accuracy')
    plt.title('Accuracy Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
best_val_acc = 0.0

# Inside your epoch loop:
val_loss, val_acc = evaluate_model(resnet101, test_loader, criterion)
val_losses.append(val_loss)
val_accuracies.append(val_acc)

if val_acc > best_val_acc:
    best_val_acc = val_acc
    torch.save(resnet101.state_dict(), "best_resnet101.pth")
    print(f"✅ New best model saved with accuracy: {val_acc:.2f}%")
